<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/tests_test_validator_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# tests/test_validator.py
import numpy as np
from rge.validator import MetaValidator, ScoreReport

class SO:
    def __init__(self, t, y, states):
        self.t, self.y, self.states = t, y, states

def test_shapes_and_basic_scoring():
    t = np.linspace(0, 1, 101)
    y = np.vstack([np.exp(-t), -np.exp(-0.8*t)])
    mv = MetaValidator()
    rep = mv.score({"equations": ["dx/dt = -x", "dy/dt = 0.8*y"]}, SO(t, y, ["x","y"]))
    assert isinstance(rep, ScoreReport)
    assert 0 <= rep.total <= 1

def test_accuracy_with_ground_truth_improves_when_closer():
    t = np.linspace(0,1,101)
    gt = np.vstack([np.exp(-t), np.exp(-t)])
    y_bad = gt + 0.5
    y_good = gt + 0.05
    mv = MetaValidator()
    rep_bad = mv.score({"equations": ["dx/dt=-x","dy/dt=-y"]}, SO(t, y_bad, ["x","y"]), ground_truth=gt)
    rep_good = mv.score({"equations": ["dx/dt=-x","dy/dt=-y"]}, SO(t, y_good, ["x","y"]), ground_truth=gt)
    assert rep_good.components["accuracy"] > rep_bad.components["accuracy"]

def test_simplicity_monotone_in_description_length():
    t = np.linspace(0,1,11)
    y = np.zeros((1,11))
    mv = MetaValidator()
    simple = mv.score({"equations": ["dx/dt=-x"]}, SO(t, y, ["x"]))
    complex_eqs = ["dx/dt=-x+sin(x)+cos(x)+x^2 - log(x) + exp(x)"]
    complex_ = mv.score({"equations": complex_eqs}, SO(t, y, ["x"]))
    assert simple.components["simplicity"] > complex_.components["simplicity"]

def test_novelty_deterministic():
    t = np.linspace(0,1,11)
    y = np.zeros((1,11))
    mv = MetaValidator(seed=123)
    rep1 = mv.score({"equations": ["dx/dt=-x"]}, SO(t, y, ["x"]))
    rep2 = mv.score({"equations": ["dx/dt=-x"]}, SO(t, y, ["x"]))
    assert rep1.components["novelty"] == rep2.components["novelty"]